## Setup the Simulator

In [33]:
%load_ext autoreload
%autoreload 2


import os
import json
import numpy as np
import sys
sys.path.append('simulation')
from unity_simulator.comm_unity import UnityCommunication
from dataset_utils import execute_script_utils as utils
from evolving_graph import scripts
from GraphReader import GraphReader, remove_nodes_from_graph, reference_graph_file, base_graph_file, init_graph_file, scene_num

print(f'Using scene {int(scene_num)-1}, i.e. TrimmedTestScene{scene_num}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using scene 1, i.e. TrimmedTestScene2


## Add necessary objects

Some objects need to be removed.

In [34]:
nodes = ['phone', 'cellphone', 'dresser', 'keyboard', 'orchid', 'pillow']
remove_nodes_from_graph(reference_graph_file, nodes, base_graph_file)

In [35]:
base_graph = GraphReader(base_graph_file)

Fill in objects placements in `AddObjects.py` with statements of the form :
 ```
 base_graph.add("food_apple", "ON", ref_nodes['dining_room']['kitchen_counter'])
 ```

### Some 'furniture' was missing in the graphs so we add them first

In [36]:
ref_nodes = base_graph.usable_nodes_by_room

washing_machine_id = base_graph.new_obj_id
base_graph.add("washing_machine", "INSIDE", ref_nodes['dining_room']['dining_room'])
base_graph.usable_nodes_by_room['dining_room']['washing_machine'] = washing_machine_id
base_graph.new_obj_id += 1

stove_id = base_graph.new_obj_id
base_graph.add("stove", "ON", ref_nodes['dining_room']['kitchen_counter'])
base_graph.usable_nodes_by_room['dining_room']['stove'] = stove_id
base_graph.new_obj_id += 1

cabinet_id = base_graph.new_obj_id
base_graph.add("kitchen_cabinet", "INSIDE", ref_nodes['dining_room']['dining_room'])
base_graph.usable_nodes_by_room['dining_room']['kitchen_cabinet'] = cabinet_id
base_graph.new_obj_id += 1

fridge_id = base_graph.new_obj_id
base_graph.add("fridge", "INSIDE", ref_nodes['dining_room']['dining_room'])
base_graph.usable_nodes_by_room['dining_room']['fridge'] = fridge_id
base_graph.new_obj_id += 1

dresser_id = base_graph.new_obj_id
base_graph.add("dresser", "INSIDE", ref_nodes['bedroom']['bedroom'])
base_graph.usable_nodes_by_room['bedroom']['dresser'] = dresser_id
base_graph.new_obj_id += 1

shoe_rack_id = base_graph.new_obj_id
base_graph.add("shoe_rack", "INSIDE", ref_nodes['dining_room']['dining_room'])
base_graph.usable_nodes_by_room['dining_room']['shoe_rack'] = shoe_rack_id
base_graph.new_obj_id += 1


base_graph.node_map['<washing_machine>'] = f'<washing_machine> ({washing_machine_id})'
base_graph.node_map['<stove>'] = f'<stove> ({stove_id})'
base_graph.node_map['<kitchen_cabinet>'] = f'<kitchen_cabinet> ({cabinet_id})'
base_graph.node_map['<fridge>'] = f'<fridge> ({fridge_id})'
base_graph.node_map['<dresser>'] = f'<dresser> ({dresser_id})'
base_graph.node_map['<shoe_rack>'] = f'<shoe_rack> ({shoe_rack_id})'

In [37]:
print('List of available locations :')
for room,nodelist in ref_nodes.items():
    print('##### ',room,' #####')
    furniture = list(nodelist.keys())
    furniture.sort()
    print(furniture)

List of available locations :
#####  bathroom  #####
['bathroom', 'bathroom_cabinet', 'bathroom_counter', 'ceilinglamp', 'curtain', 'drawing', 'faucet', 'light', 'mat', 'shower', 'sink', 'toilet', 'walllamp']
#####  dining_room  #####
['bench', 'bookshelf', 'ceilinglamp', 'coffe_maker', 'cupboard', 'dining_room', 'dishwasher', 'drawing', 'faucet', 'freezer', 'fridge', 'kitchen_cabinet', 'kitchen_counter', 'knifeblock', 'light', 'mat', 'microwave', 'oven', 'pot', 'powersocket', 'shoe_rack', 'sink', 'stove', 'stovefan', 'table', 'toaster', 'trashcan', 'tray', 'tvstand', 'walllamp', 'washing_machine']
#####  bedroom  #####
['bed', 'bedroom', 'bookshelf', 'ceilinglamp', 'chair', 'computer', 'cpuscreen', 'curtain', 'desk', 'drawing', 'dresser', 'filing_cabinet', 'light', 'mat', 'mouse', 'mousepad', 'nightstand', 'photoframe', 'sofa', 'table', 'tablelamp']
#####  home_office  #####
['bookshelf', 'ceilinglamp', 'chair', 'computer', 'cpuscreen', 'curtain', 'desk', 'drawing', 'hanger', 'home_of

In [38]:
%run -i 'AddObjects.py'

In [39]:
base_graph.write(init_graph_file)

## Write the Program

In [40]:
from ProgramExecutor import ProgramExecutor

### Write the desired program in `program.txt`. The program should include a list of actions and comments specifying how much time you expect it to take
Example
```
## 1-2 mins
[Walk] <dining_room>
[Walk] <food_apple>
## 3-4 mins
[Grab] <food_apple>
[Walk] <kitchen_counter>
[PutBack] <food_apple> <kitchen_counter>
```

Available Actions : 

`WALK, FIND, SIT, STANDUP, GRAB, OPEN, CLOSE, PUTBACK, PUTIN, SWITCHON, SWITCHOFF, DRINK, LOOKAT, TURNTO, WIPE, RUN, PUTON, PUTOFF, GREET, DROP, READ, POINTAT, TOUCH, LIE, PUTOBJBACK, POUR, TYPE, WATCH, PUSH, PULL, MOVE, RINSE, WASH, SCRUB, SQUEEZE, PLUGIN, PLUGOUT, CUT, EAT, SLEEP, WAKEUP, RELEASE`

In [41]:
init_graph = GraphReader(graph_file=init_graph_file)

Activity options : 

getting_out_of_bed, brushing_teeth, showering, getting_dressed, leave_home, going_to_the_bathroom, breakfast, lunch, dinner, reading, computer_work, watching_tv, hand_wash_clothes, laundry, kitchen_cleaning, vaccuum_cleaning, wash_dishes, take_out_trash, taking_medication, sleeping, cleaning, listening_to_music, socializing, diary_logging, playing_music, coming_back_home

In [43]:
activity = input('Which are you scripting?')
directory = os.path.join('sourcedScripts',activity)
if not os.path.exists(directory):
    raise KeyError('Are you sure you typed the correct activity? Please try again...')

In [44]:
with open('ActivityObjects.json') as f:
    available_obj_for_activity = json.load(f)[activity]

for room, stuff in init_graph.expanded_nodes_by_room.items():
    available_obj = [full_obj for full_obj,(obj,id) in list(stuff.items()) if obj in available_obj_for_activity]
    available_obj.sort()
    print('##### ',room,' #####')
    print(available_obj)

#####  bathroom  #####
[]
#####  dining_room  #####
['ice', 'napkin', 'tray', 'wine', 'wine_glass']
#####  bedroom  #####
['cellphone', 'chair_bedroom', 'sofa_bedroom']
#####  home_office  #####
['board_game', 'chair_home_office', 'chessboard', 'deck_of_cards', 'phone', 'scrabble', 'sofa_home_office', 'telephone', 'video_game_console', 'video_game_controller']


## Please write your script in `program.txt` now
Run the cell below to check if it works

In [45]:
action_headers, graphs = ProgramExecutor(program_file='program.txt', graph_file=init_graph_file, node_map=init_graph.node_map).execute()

## If everything looks good, move the script to your database :)

In [46]:
existing_files = [int(os.path.splitext(file)[0]) for file in os.listdir(directory)]
if existing_files:
    new_file = os.path.join(directory, str(max(existing_files)+1)+'.txt')
else:
    new_file = os.path.join(directory, '0.txt')
print(f'Saving your awesome script at {new_file}')
os.rename('program.txt',new_file)
open('program.txt', 'a').close()

Saving your awesome script at sourcedScripts/socializing/0.txt


### In case you need to remove your script

In [47]:
# os.remove(new_file)

# Hurray! You did it :)